In [47]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [48]:
dataset = 'emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [5] # [1, 5, 10, 20]
metrics = ['nDCG'] # 'Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']
results_dir

'../results/emma/performance/'

In [62]:
def find_modality(string, keyword="modalites="):
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [64]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        file_path = results_dir + f
        df_tmp = pd.read_csv(file_path, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        if 'SiBraR' in df_tmp['model'].values[0]:
            keyword = "modalities="
        else:
            keyword = "modalites="
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0], keyword=keyword)
        model = df_tmp['model'].str.split('_').str[0]
        df_tmp['model'] = model

        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)
        # print(f)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # merge with previous cutoffs
    if not df.empty:
        # print(df_cut)
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        # print(df.shape)
    else:
        df = df_cut
        # print(df.head())

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['nDCG@5']]

/tmp/ipykernel_105233/1828473671.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('nDCG@5', inplace=True)


,model,modality,date,nDCG@5
0,SiBraR,audio,2025_02_21_15_41_51,0.253716
0,DeepMF,None,2025_02_20_15_58_42,0.278062
0,VBPR,audio,2025_02_21_18_02_30,0.278407
0,LightGCNM,audio,2025_02_21_17_31_08,0.291294
0,FeatureItemKNN,emotion,2025_02_20_09_56_51,0.303772
0,ItemKNN,None,2025_02_21_15_17_19,0.307664


In [ ]:
# export to comma seperated file
# df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [ ]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)